Привет, меня зовут Артем Хуршудов. Сегодня я проверю твой проект.
<br> Дальнейшее общение будет происходить на "ты" если это не вызывает никаких проблем.
<br> Желательно реагировать на каждый мой комментарий ('исправил', 'не понятно как исправить ошибку', ...)
<br> Пожалуйста, не удаляй комментарии ревьюера, так как они повышают качество повторного ревью.

Комментарии будут в <font color='green'>зеленой</font>, <font color='blue'>синей</font> или <font color='red'>красной</font> рамках:

<div class="alert alert-block alert-success">
<b>Успех:</b> Если все сделано отлично
</div>

<div class="alert alert-block alert-info">
<b>Совет: </b> Если можно немного улучшить
</div>

<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Если требуются исправления. Работа не может быть принята с красными комментариями.
</div>

-------------------

Будет очень хорошо, если ты будешь помечать свои действия следующим образом:
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> ...
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Были внесены следующие изменения ...
</div>

### <font color='orange'>Общее впечатление</font>
* Большое спасибо за проделанную работу. Видно, что приложено много усилий.
* Выводы и рассуждения получились содержательными, их было интересно читать.
* Радует, что тебе удалось разобраться с техникой бутстрапа.
* Отлично, что доверительный интервал и риск убытков посчитаны верно.
* К сожалению, над проектом нужно будет еще немного поработать. Однако, правки не должны занять много времени.
* В работе ты сможешь найти несколько советов. Буду рад, если ты обратишь на них внимание.
* С радостью отвечу на твои вопросы, если они есть. Лучше всего их собрать в следующей ячейке. Жду новую версию проекта.

# Машинное обучение в бизнесе
____

## Описание проекта
Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.
Шаги для выбора локации обычно такие:
* В избранном регионе собирают характеристики для скважин: качество нефти и объём её запасов;
* Строят модель для предсказания объёма запасов в новых скважинах;
* Выбирают скважины с самыми высокими оценками значений;
* Определяют регион с максимальной суммарной прибылью отобранных скважин.

Вам предоставлены пробы нефти в трёх регионах. Характеристики для каждой скважины в регионе уже известны. Постройте модель для определения региона, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой Bootstrap.
___

## Описание данных
* **Признаки:**
    1. **`id`** — уникальный идентификатор скважины;
    2. **`f0`**, **`f1`**, **`f2`** — три признака точек;
* **Целевой признак:**
    1. **`product`** — объём запасов в скважине (тыс. баррелей).
___

## Структура проекта
<a href='#section1'></a>
1. [Подготовка данных](#section1)
<a href='#section2'></a>
2. [Обучение моделей в регионах](#section2)
<a href='#section3'></a>
3. [Метрика прибыли](#section3)
<a href='#section4'></a>
4. [Рассчёт прибыльности регионов](#section4)
<a href='#section5'></a>
5. [Выводы исследования](#section5)
___
___

<a id='section1'></a>
## 1. Подготовка данных

In [1]:
import numpy as np
import pandas as pd
import warnings 
# Модели
from sklearn.linear_model import LinearRegression
# Интструменты
from sklearn.model_selection import train_test_split as tts
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.utils import shuffle
from tqdm import tqdm
# Графики
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings("ignore")

<div class="alert alert-block alert-success">
<b>Успех:</b> Отлично, что все импорты собраны в первой ячейке ноутбука! Если у того, кто будет запускать твой ноутбук будут отсутствовать некоторые библиотеки, то он это увидит сразу, а не в процессе!
</div>

In [2]:
geo_data_0 = pd.read_csv('/datasets/geo_data_0.csv')
geo_data_1 = pd.read_csv('/datasets/geo_data_1.csv')
geo_data_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
# Первый взгляд.
hatched_line = '\n' + 100*'-' + '\n'
bold_line = 100*'—'
for data in [geo_data_0, geo_data_1, geo_data_2]:
    print(
        bold_line, '\n',
        data.info(), hatched_line,
        'HEAD:\n', data.head(), hatched_line,
        'DESCRIBE:\n', data.describe(), hatched_line,
        'NaNs percent:\n', (data.isna().sum() / data.shape[0] * 100).round(2).sort_values(),
        'Duplicates:\n', data.duplicated().sum(),
        bold_line, '\n',
        sep='\n'
    )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
————————————————————————————————————————————————————————————————————————————————————————————————————


None

----------------------------------------------------------------------------------------------------

HEAD:

      id        f0        f1        f2     product
0  txEyH  0.705745 -0.497823  1.221170  105.280062
1  2acmU  1.334711 -0.340164  4.365080   73.037750
2  409Wp  1.022732  0.151990  1.419926   85.265647
3  iJLyR -0.032172  0.139033  2.978566  168.620776
4  Xdl7t  1.988431  0.155413  4.751769  154.036647

----------------------------------------------------------------------------------------------------

DESCRIBE:

                  f0       

<div class="alert alert-block alert-success">
<b>Успех:</b> Загрузка данных и их первичный осмотр проведены корректно.
</div>

`NaN`s не наблюдаем в данных, однако во время `data.describe()` вылезли нули (минимумы значений в столбце `product`. Рассмотрим их.

In [4]:
for i, data in enumerate([geo_data_0, geo_data_1, geo_data_2]):
    zeros_num = data[data['product']==0]['product'].count()
    zeros_percent = zeros_num / data.shape[0]
    print(
        'Количество нулевых запасов в скважинах региона {:.0f} : {:.0f}'.format(i, zeros_num), '\n'
        'Процент нулевых запасов в скважинах региона {:.0f} : {:.2%}'.format(i, zeros_percent)
    )
    print()

Количество нулевых запасов в скважинах региона 0 : 1 
Процент нулевых запасов в скважинах региона 0 : 0.00%

Количество нулевых запасов в скважинах региона 1 : 8235 
Процент нулевых запасов в скважинах региона 1 : 8.24%

Количество нулевых запасов в скважинах региона 2 : 1 
Процент нулевых запасов в скважинах региона 2 : 0.00%



Наблюдается отличие второго региона от своих "конкурентов"😉. Либо имеет ошибка в вызгрузке данных, либо работники поленились измерить (маловероятно), либо не успели измерить до конца к моменту выгрузки данных, либо имеют место уже опустощённые скважины в регионе. 

В силу невозможности их классифицировать (прогуляться до заказчика / отдела в компании / сборщиков данных и т.д. и задать вопросы), нулевые данные во втором регионе остаются.

Исбавимся от столбца `id`. Никакой полезной для обучения информации не несёт. 

In [5]:
geo_data_0 = geo_data_0.drop(labels=['id'], axis=1)
geo_data_1 = geo_data_1.drop(labels=['id'], axis=1)
geo_data_2 = geo_data_2.drop(labels=['id'], axis=1)

**Выводы:**
1. Загруженные данные во всех регионах без пропусков;
2. Во втором регионе замечены скважины с нулевым объёмом ($8.24\%$). Было решено их оставить из-за невозможности классифицировать проблему в достаточной мере, чтобы удалять.
___

<div class="alert alert-block alert-success">
<b>Успех:</b> Согласен, что по поводу этих странных значений стоит проконсультироваться со специалистами.
</div>

<a id='section2'></a>
## 2. Обучение моделей в регионах

**План действий:**
1. Нормализация признаков для стабильности работы моделей;
2. Разбиение на обучающую и валидационные выборки;
3. Обучение модели `Линейной регрессии`.

Нормализацию проводим с помощью `StandardScaler`.

In [6]:
def train_valid(data, target_name='product', valid_size=0.25, is_scaled=True):
    target = data[target_name]
    features = data.drop([target_name], axis=1)
    
    features_train, features_valid, target_train, target_valid = tts(
        features, target, test_size=valid_size, random_state=271828
    )
    numeric = features.columns
    features_list = [features_train, features_valid]
    def scaler(features_list, numeric):
        scaler = StandardScaler()
        scaler.fit(features[numeric])
        features_list[0][numeric] = scaler.transform(features_list[0][numeric])
        features_list[1][numeric] = scaler.transform(features_list[1][numeric])
        return features_list
    features_list = scaler(features_list, numeric)
    return features_list[0], features_list[1], target_train, target_valid

In [7]:
def print_table(table):
    longest_cols = [
        (max([len(str(row[i])) for row in table]) + 3)
        for i in range(len(table[0]))
    ]
    row_format = "".join(["{:>" + str(longest_col) + "}" for longest_col in longest_cols])
    for row in table:
        print(row_format.format(*row))

In [8]:
# Инициализируем модель Линейной регрессии.
model = LinearRegression()

In [9]:
def get_predictions(data_t : tuple()):
    features_train, features_valid, target_train, target_valid = data_t
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    df = pd.DataFrame(
        data = {
            'product' : target_valid,
            'predicted' : predicted_valid
               },
        columns = ['product', 'predicted']
    )
    rmse = np.round(mean_squared_error(target_valid, predicted_valid) ** .5, 4)
    mae = np.round(mean_absolute_error(target_valid, predicted_valid), 4)
    r2 = np.round(r2_score(target_valid, predicted_valid), 4)
    product_mean = np.round(df['product'].mean(), 4)
    predicted_mean = np.round(df['predicted'].mean(), 4)
    product_sum = np.round(df['product'].sum(), 2)
    predicted_sum = np.round(df['predicted'].sum(), 2)
    params_df = pd.DataFrame(
        data = {
            'product_mean' : product_mean,
            'predicted_mean' : predicted_mean,
            'product_sum' : product_sum,
            'predicted_sum' : predicted_sum
        },
        columns = ['product_mean', 'predicted_mean', 'product_sum', 'predicted_sum'],
        index = [0]
    )
    metrics = [rmse, mae, r2]
    return df, params_df, metrics

**`geo_data_0`**

In [10]:
predictions_0, params_0, metrics_0 = get_predictions(train_valid(geo_data_0))
print('RMSE модели для региона 0 =', metrics_0[0])
print('MAE модели для региона 0 =', metrics_0[1])
print('R^2 модели для региона 0 =', metrics_0[2])
predictions_0.sample(5)

RMSE модели для региона 0 = 37.6225
MAE модели для региона 0 = 31.0734
R^2 модели для региона 0 = 0.2806


,product,predicted
38389,25.526870,83.766632
31161,131.154707,104.211403
73132,98.535996,96.622358
13214,137.683489,77.075018
89407,132.018640,72.702922


**`geo_data_1`**

In [11]:
predictions_1, params_1, metrics_1 = get_predictions(train_valid(geo_data_1))
print('RMSE модели для региона 1 =', metrics_1[0])
print('MAE модели для региона 1 =', metrics_1[1])
print('R^2 модели для региона 1 =', metrics_1[2])
predictions_1.head(5)

RMSE модели для региона 1 = 0.8889
MAE модели для региона 1 = 0.7168
R^2 модели для региона 1 = 0.9996


,product,predicted
65242,30.132364,29.949185
11433,30.132364,29.840978
3552,84.038886,82.753300
37035,84.038886,83.074599
28297,30.132364,30.074786


**`geo_data_2`**

In [12]:
predictions_2, params_2, metrics_2 = get_predictions(train_valid(geo_data_2))
print('RMSE модели для региона 1 =', metrics_2[0])
print('MAE модели для региона 1 =', metrics_2[1])
print('R^2 модели для региона 1 =', metrics_2[2])
predictions_2.head(5)

RMSE модели для региона 1 = 40.1322
MAE модели для региона 1 = 32.9593
R^2 модели для региона 1 = 0.2036


,product,predicted
65242,93.287617,84.200262
11433,128.822066,86.067169
3552,8.391635,81.086119
37035,74.060369,76.811163
28297,140.158561,76.418823


Таблицы реальных/предсказанных средних и сумм запасов сырья в регионе с RMSE моделей, получивших такие результаты.

In [13]:
metrics = pd.DataFrame(data = [metrics_0, metrics_1, metrics_2], columns = ['RMSE', 'MAE', 'R^2'])
params = pd.concat([params_0, params_1, params_2], ignore_index=True)
final_table = pd.concat([params, metrics], axis=1)
final_table

,product_mean,predicted_mean,product_sum,predicted_sum,RMSE,MAE,R^2
0,92.5533,92.4899,2313832.65,2312248.46,37.6225,31.0734,0.2806
1,68.9755,68.9867,1724387.12,1724666.84,0.8889,0.7168,0.9996
2,94.8377,94.9058,2370943.26,2372644.44,40.1322,32.9593,0.2036


**Результаты:**
1. Построенные модели для каждого региона предсказывают с высокой долей точности (<< $1\%$) суммарные запасы в регионе.
2. Наилучший показатель по $RMSE, MAE$ у `региона_1`, однако согласно коэффициенту Пирсона, модель сильно переобучилась. Вдобавок помним об утерянных $8,24\%$ данных и сумма запасов в этом регионе наименьшая из всех.
3. Из вышесказанного конечный выбор региона для разработки будет производить между `регионом_0` и `регионом_2`.

<div class="alert alert-block alert-success">
<b>Успех:</b> Модели обучены корректно. Радует, что использованы функции, это позволяет избежать дублирования кода.
</div>

<a id='section3'></a>
## 3. Метрика прибыли

Вводные данные

In [14]:
BUDGET = 1e7    # [тыс. рублей]
CHOSEN_POINTS = 500
BEST_POINTS = 200
PRICE_PER_BARREL = 450

<div class="alert alert-block alert-info">
<b>Совет: </b> В названиях константных переменных лучше использовать только БОЛЬШИЕ буквы. Это соглашение между программистами.
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> Исправлено. Согласен, так понятнее.
</div>

Рассчитаем безубыточные характеристики для разработки на лучших скважинах.

In [16]:
average_revenue_per_point = BUDGET / BEST_POINTS
print('Средняя прибыль с лучших точек = ', np.round(average_revenue_per_point), '[тыс. рублей]')
average_barrels_per_point = average_revenue_per_point / PRICE_PER_BARREL
print('Средняя объём с лучших точек = ', np.round(average_barrels_per_point, 2), '[тыс. рублей]')
average_barrels = BUDGET / PRICE_PER_BARREL
print('Средняя объём в регионе = ', np.round(average_barrels, 2), '[тыс. рублей]')
print(
    'Средние запасы в регионах (по порядку):', 
    np.round(geo_data_0['product'].mean(), 4), 
    np.round(geo_data_1['product'].mean(), 4), 
    np.round(geo_data_2['product'].mean(), 4)
     )

Средняя прибыль с лучших точек =  50000.0 [тыс. рублей]
Средняя объём с лучших точек =  111.11 [тыс. рублей]
Средняя объём в регионе =  22222.22 [тыс. рублей]
Средние запасы в регионах (по порядку): 92.5 68.825 95.0


**Выводы:**
1. Рассчитаны безубыточные характеристики для разработки на лучших скважинах;
2. По оценке средних суммарных запасов в регионах наиболее предпочтительными (повторно) кажутся `регион_0`, `регион_2`.

<div class="alert alert-block alert-success">
<b>Успех:</b> Точка безубыточности найдена верно, сравнение проведено.
</div>

<a id='section4'></a>
## 4. Расчёт прибыльности регионов

In [17]:
state = np.random.RandomState(271828)

In [22]:
def bootstrap(target, probabilities, steps=1000):
    values = list()
    
    def get_profit(target, probabilities, count):
        probabilities_sorted = probabilities.sort_values(ascending=False)
        selected = target[probabilities_sorted.index][:count]
        return PRICE_PER_BARREL * selected.sum() - BUDGET
        
    for i in tqdm(range(steps)):
        target_sample = target.sample(n=CHOSEN_POINTS, replace=True, random_state=state)
        probabilities_sample = probabilities[target_sample.index]
        
        profit = get_profit(target, probabilities_sample, BEST_POINTS)
        values.append(profit)
    
    values = pd.Series(values)
    lower = values.quantile(0.025)
    upper = values.quantile(0.975)
    mean = values.mean()
    risk = (values < 0).mean()
    return mean, lower, upper, risk

<div class="alert alert-block alert-info">
<b>Совет: </b> Определение функции get_revenue можно было внести за функцию bootstrap. Также, корректнее было бы назвать get_profit, так как revenue – это выручка.
</div>
<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Функция get_profit планировалась к использованию только в процесс бутстрепа, поэтому и положил её внутрь. 
    1. Она являет собой функцию действия над подвыборками для получения значений, т.е. является частью алгоритма, поэтому посчитал логичным инициализировать внутри алгоритма (функция bootstrap в данном случае).
    2. Вторым соображением была оценка по средним и отдельная функция для этого. В ходе работы от идеи отказался, но переписывать лень, потому что есть соображение (1) 😉.
</div>

<div class="alert alert-block alert-warning">
<b>Изменения:</b> С названиями согласен, изменил. Спасибо!
</div>

<div class="alert alert-block alert-info">

<b>Совет: </b> Риск можно было найти проще: `(values < 0).mean()`.
</div>
    
<div class="alert alert-block alert-warning">
<b>Изменения:</b> 👍
</div>

**`geo_data_0`**

In [23]:
mean_0, lower_0, upper_0, risk_0 = bootstrap(predictions_0['product'], predictions_0['predicted'], steps=1000)
print(
    'Регион 0:\n',
    'Средняя прибыль = {:.2f} [тыс. рублей]'.format(mean_0),
    'Доверительный интервал 95% = ( {:.2f} , {:.2f} ) [тыс. рублей]'.format(lower_0, upper_0),
    'Риск разработки = {:.2%}'.format(risk_0),
    sep='\n'
)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 260.36it/s]

Регион 0:

Средняя прибыль = 456196.01 [тыс. рублей]
Доверительный интервал 95% = ( -46978.39 , 962055.09 ) [тыс. рублей]
Риск разработки = 3.70%


**`geo_data_1`**

In [24]:
mean_1, lower_1, upper_1, risk_1 = bootstrap(predictions_1['product'], predictions_1['predicted'], steps=1000)
print(
    'Регион 1:\n',
    'Средняя прибыль = {:.2f} [тыс. рублей]'.format(mean_1),
    'Доверительный интервал 95% = ( {:.2f} , {:.2f} ) [тыс. рублей]'.format(lower_1, upper_1),
    'Риск разработки = {:.2%}'.format(risk_1),
    sep='\n'
)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:04<00:00, 249.54it/s]

Регион 1:

Средняя прибыль = 461144.99 [тыс. рублей]
Доверительный интервал 95% = ( 75982.67 , 847330.57 ) [тыс. рублей]
Риск разработки = 1.20%


**`geo_data_2`**

In [25]:
mean_2, lower_2, upper_2, risk_2 = bootstrap(predictions_2['product'], predictions_2['predicted'], steps=1000)
print(
    'Регион 2:\n',
    'Средняя прибыль = {:.2f} [тыс. рублей]'.format(mean_2),
    'Доверительный интервал 95% = ( {:.2f} , {:.2f} ) [тыс. рублей]'.format(lower_2, upper_2),
    'Риск разработки = {:.2%}'.format(risk_2),
    sep='\n'
)

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:03<00:00, 254.78it/s]

Регион 2:

Средняя прибыль = 382547.76 [тыс. рублей]
Доверительный интервал 95% = ( -143215.27 , 905934.67 ) [тыс. рублей]
Риск разработки = 7.30%


**Результаты:**
1. Для всех трёх регионов были рассчитаны экономические характеристики: прибыль, доверительный интервал, риски;
2. Наименьшие риски показывает `регион_1`. Даже при самом неудачном выборе точек нижняя граница интервала позволит остаться разработке доходной;
3. Далее по степени риска идут `регион_0`, `регион_2` соответственно.

<div class="alert alert-block alert-success">
<b>Успех:</b> В целом, средняя прибыль и другие статистики посчитаны абсолютно корректно. Радует, что для доверительного интервала использовался метод квантилей – это единственно верное решение.
</div>

<a id='section5'></a>
## 5. Выводы исследования

1. **Подготовка данных:**  
    1. Загруженные данные во всех регионах без пропусков;
    2. Во втором регионе замечены скважины с нулевым объёмом ($8.24\%$). Было решено их оставить из-за невозможности классифицировать проблему в достаточной мере, чтобы удалять.
2. **Обучение моделей в регионах:**  
    1. Построенные модели для каждого региона предсказывают с высокой долей точности (<< $1\%$) суммарные запасы в регионе.
    2. Наилучший показатель по $RMSE, MAE$ у `региона_1`, однако согласно коэффициенту Пирсона, модель сильно переобучилась. Вдобавок помним об утерянных $8,24\%$ данных и сумма запасов в этом регионе наименьшая из всех.
    3. Из вышесказанного конечный выбор региона для разработки будет производить между `регионом_0` и `регионом_2`.
3. **Метрика прибыли:**  
    1. Рассчитаны безубыточные характеристики для разработки на лучших скважинах;
    2. По оценке средних суммарных запасов в регионах наиболее предпочтительными (повторно) кажутся `регион_0`, `регион_2`.
4. **Расчёт прибыльности регионов:**  
    1. Для всех трёх регионов были рассчитаны экономические характеристики: прибыль, доверительный интервал, риски;
    2. Наименьшие риски показывает `регион_1`. Даже при самом неудачном выборе точек нижняя граница интервала позволит остаться разработке доходной;
    3. Далее по степени риска идут `регион_0`, `регион_2` соответственно.
5. **Общий вывод:**  
В результате проведённого исследования для разработки рекомендуется взять `регион_1`. 
    * Он единственный имеет достаточно малую степень риска ($< 2,5\%$), что компенсирует замено меньшие суммарные запасы;


<div class="alert alert-block alert-danger">
<b>Ошибка:</b> Регион выбран некорректно, так как в конце задания четко сформулирован алгоритм: <i>После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.</i>.
</div>

<div class="alert alert-block alert-warning">
<b>Комментарий студента:</b> Забыл про это...
    Исправлено.
</div>